In [127]:
import pandas as pd
import os
import numpy as np


%matplotlib inline
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.width', 1000)

from IPython.core.display import display, HTML
display(HTML("<style>"
    + "#notebook { padding-top:0px !important; } " 
    + ".container { width:100% !important; } "
    + ".end_space { min-height:0px !important; } "
    + "</style>"))


In [416]:
event_meta = {
    'onset':{'Description': "Time the draw command on the monitor starts in seconds in reference to the scan start time of the block. Negative times indicate onsets before the first scanning block."},
    'onset_delay':{'Description': "Difference between onset time and intended display time"},
    'onset_unadjusted':{'Description': "True onset time in seconds without subtracting scanner start time"},
    'amount_change':{'Description': "Change in winnings"},
    'certain_amount':{'Description': "Value of the non-gamble choice"},
    'current_amount':{'Description': "Current winnings"},
    'duration': {'Description':'Time in seconds that this phase of the trial is presented'},
    'mood_target':{'Description': "Targeted mood value of the block, higher value represents happier mood"},
    'multiplier':{'Description': ""},
    'outcome':{'Description': "Outcome of the trial - result of the gamble or certain"},
    'outcome1':{'Description': "Value of outcome 1"},
    'outcome2':{'Description': "Value of outcome 2"},
    'question_duration':{'Description': "Duration of question in mood rating trials"},
    'rating':{'Description': "Whether or not the participant was asked to rate their mood"},
    'response':{'Description': "Value of the key response. 2 is gamble, 4 is certain outcome."},
    'rpe':{'Description': "Model-based reward prediction error"},
    'rt': {'Description': "Reaction time in seconds since onset time"},
    'rt_time': {'Description': "Time of the response in seconds since the start of the scan"},
    'slider_response':{'Description': "Value of slider response for mood ratings - higher value represents happier mood"},
    'trial_phase': {'Description': 'Which phase of the trial the event represents'},
    'trial_type': {'Description': 'Categorization of the trial on the basis of the cue type and the result'}
}

In [431]:
def eprime_to_bids(fd):
    #Review these event types
    ev_types = ['Welcome', 'BlockHappyPreview', 'BlockStartHappiness','ScanWait', 'GetReady', 'GetAnswer', 'ShowChoice', 'ShowResult', 'TrialHappiness','ISIRate','ISITrial', 'CurrentEarnings', 'LifeHappyPreview', 'LifeHappiness', 'Rest', 'Goodbye']
    time_types = {'onset': 'OnsetTime',
                  'onset_delay': 'OnsetDelay',
                  'duration': 'Duration',
                  'rt_time': 'RTTime',
                  'rt': 'RT',
                  'response': 'RESP',
                  'slider_response': 'SliderResp'
                 }

    #Times to be adjusted by subtracting the scanner start time
    adj_times = ['onset', 'rt_time']
    
    for file in os.listdir(fd + 'MMI_events/'):
        #iterates through all csv and excel files
        if '.csv' in file:
            task_events = pd.read_csv(f'{fd}MMI_events/{file}')
        elif '.x' in file:
            task_events = pd.read_excel(f'{fd}MMI_events/{file}')
        else:
            print("Unable to read events file.")
         
        #Gets subject number from file
        sub = file.split('-')[1].split('_')[0]

        #Gets scan start times for each block
        scan_wait_offsets = task_events[np.isnan(task_events.ScanWait__OffsetTime) == False].ScanWait__OffsetTime.tolist()

        events = []
        block = 0

        for ix, te_row in task_events.iterrows():
            for phase in ev_types:
                ev = {}
                for pyname, epname in time_types.items():
                    try: 
                        #subtract scan wait offset times from nonzero onset and reaction times
                        if pyname in adj_times and te_row[f'{phase}__{epname}'] > 0:
                            #determines block to subtract appropriate scanwait offset time
                            zero = False
                            if te_row[f'{phase}__{epname}'] <= scan_wait_offsets[0]:
                                block = 0
                                zero = True
                            elif te_row[f'{phase}__{epname}'] <= scan_wait_offsets[1]:
                                block = 1
                            elif te_row[f'{phase}__{epname}'] <= scan_wait_offsets[2]:
                                block = 2
                            else:
                                block = 3
                            ev['block'] = block
                            if not zero: block -= 1
                            ev[pyname] = te_row[f'{phase}__{epname}'] - scan_wait_offsets[block]
                        #if duration is 0, uses the last motion of the slider as duration
                        elif pyname == 'duration' and te_row[f'{phase}__{epname}'] == 0.0:
                            ev[pyname] = te_row[f'{phase}__Slider_LastMotion']
                        #all other columns assigned without adjustment
                        else:
                            ev[pyname] = te_row[f'{phase}__{epname}']
                    except KeyError:
                        pass
                #Adds trial number from TrialList Sample
                ev['trial_number'] = te_row['TrialList__Sample']
                ev['onset_unadjusted'] = te_row[f'{phase}__OnsetTime']
                ev['trial_phase'] = phase
                ev['trial_type'] = te_row['TrialType']
                ev['rpe'] = te_row['RPE']
                ev['outcome'] = te_row['Outcome']
                ev['outcome1_amount'] = te_row['Outcome1Amount']
                ev['outcome2_amount'] = te_row['Outcome2Amount']
                ev['certain_amount'] = te_row['CertainAmount']
                ev['current_amount'] = te_row['CurrentAmount']
                ev['amount_change'] = te_row['AmountChange']
                ev['rating'] = te_row['DoRating']
                ev['multiplier'] = te_row['Multiplier']
                ev['question_duration'] = te_row['QuestionDur']
                ev['mood_target'] = te_row['MoodTarget']
                events.append(ev)    

        events = pd.DataFrame(events)

        #drop rows without an onset (or if onset is 0)
        events = events[events.onset != 0]
        events.dropna(subset=['onset'], inplace=True)

        #convert reaction times of 0 to NaN
        events.loc[events.rt_time== 0, 'rt'] = np.nan
        events.loc[events.rt_time== 0, 'rt_time'] = np.nan

        #convert block column to int
        events.block = events.block.astype({"block":int})

        # cast time fields to seconds from ms
        time_fields =['duration', 'onset','onset_delay',
                      'onset_unadjusted','rt','rt_time'
                     ]
        events.loc[:, time_fields] = events.loc[:, time_fields] / 1000

        #Sort by trial number and onset time, adjust column order
        events = events.sort_values(['block','onset']).reset_index(drop=True)
        cols = events.columns.tolist()
        cols.remove('onset')
        cols.remove('onset_delay')
        cols.remove('onset_unadjusted')
        cols.remove('trial_number')
        cols.remove('block')
        cols = ['trial_number','block','onset','onset_delay','onset_unadjusted'] + cols
        events = events[cols]
        events.to_csv(f'{fd}BIDS_events/BIDS_MMI_events_{sub}.tsv',sep='\t',index=False)

In [432]:
events = eprime_to_bids('./')

## Debugging

In [387]:
#Working function with debug statements and columns
def eprime_to_bids(fd):
    #Review these event types
    ev_types = ['Welcome', 'BlockHappyPreview', 'BlockStartHappiness','ScanWait', 'GetReady', 'GetAnswer', 'ShowChoice', 'ShowResult', 'TrialHappiness','ISIRate','ISITrial', 'CurrentEarnings', 'LifeHappyPreview', 'LifeHappiness', 'Rest', 'Goodbye']
    time_types = {'onset': 'OnsetTime',
                  'onset_delay': 'OnsetDelay',
                  'duration': 'Duration',
                  'rt_time': 'RTTime',
                  'rt': 'RT',
                  'response': 'RESP',
                  'slider_response': 'SliderResp'
                 }

    #getting scan wait offset times for each block to subtract from onset and reaction times within the indices of each block
    adj_times = ['onset', 'rt_time']
    for file in os.listdir(fd + 'MMI_events/'):
        #iterates through all csv and excel files
        if '.csv' in file:
            task_events = pd.read_csv(f'{fd}MMI_events/{file}')
        elif '.x' in file:
            task_events = pd.read_excel(f'{fd}MMI_events/{file}')
        else:
            print("Unable to read events file.")
            
        sub = file.split('-')[1].split('_')[0]

        scan_wait_offsets = task_events[np.isnan(task_events.ScanWait__OffsetTime) == False].ScanWait__OffsetTime.tolist()

        events = []
        block = 0

        for ix, te_row in task_events.iterrows():
            for phase in ev_types:
                ev = {}
                for pyname, epname in time_types.items():
                    try: 
                        #subtract offset times from nonzero onset and reaction times
                        if pyname in adj_times and te_row[f'{phase}__{epname}'] > 0:
                            #determines block to subtract appropriate scanwait offset time
                            if te_row[f'{phase}__{epname}'] <= scan_wait_offsets[1]:
                                block = 0
                            elif te_row[f'{phase}__{epname}'] <= scan_wait_offsets[2]:
                                block = 1
                            else:
                                block = 2
                            ev['block'] = block
                            ev[pyname] = te_row[f'{phase}__{epname}'] - scan_wait_offsets[block]
                            ev[f'{pyname}_unadj'] = te_row[f'{phase}__{epname}']
                            ev['scan_wait_offset'] = scan_wait_offsets[block]
                            var = phase+'__'+epname
                            print(f'Subtracting {scan_wait_offsets[block]} from {te_row[var]} ({ev[pyname]}). pyname:{pyname}, epname:{epname}, phase:{phase}, ix: {ix}')
                        elif pyname == 'duration' and te_row[f'{phase}__{epname}'] == 0.0:
                            ev[pyname] = te_row[f'{phase}__Slider_LastMotion']
                            print(f'***********duration****************** ({ev[pyname]}). pyname:{pyname}, epname:{epname}, phase:{phase}, ix: {ix}')
                        else:
                            ev[pyname] = te_row[f'{phase}__{epname}']
                            var = phase+'__'+epname
                            print(f'{te_row[var]} less than or equal to zero. pyname:{pyname}, epname:{epname}, phase:{phase}, ix: {ix}')
                    except KeyError:
                        #Calculates duration of response phases using last slider motion
                        #Should onset time be SliderResp or regular onset for these phases?
                        print(f'KeyError. pyname:{pyname}, epname:{epname}, phase:{phase}, ix: {ix}')
                        pass
                #Adds trial number from TrialList Sample, converts float to int 
                try:
                    ev['trial_number'] = int(te_row['TrialList__Sample'])
                except ValueError:
                    pass
                ev['trial_phase'] = phase
                ev['trial_type'] = te_row['TrialType']
                ev['rpe'] = te_row['RPE']
                ev['outcome'] = te_row['Outcome']
                ev['outcome1_amount'] = te_row['Outcome1Amount']
                ev['outcome2_amount'] = te_row['Outcome2Amount']
                ev['certain_amount'] = te_row['CertainAmount']
                ev['current_amount'] = te_row['CurrentAmount']
                ev['amount_change'] = te_row['AmountChange']
                ev['rating'] = te_row['DoRating']
                ev['multiplier'] = te_row['Multiplier']
                ev['question_duration'] = te_row['QuestionDur']
                ev['mood_target'] = te_row['MoodTarget']
                events.append(ev)    

        events = pd.DataFrame(events)

        #drop rows without an onset (or if onset is 0)
        events = events[events.onset != 0]
        events.dropna(subset=['onset'], inplace=True)

        #convert reaction times of 0 to NaN
        events.loc[events.rt_time== 0, 'rt'] = np.nan
        events.loc[events.rt_time== 0, 'rt_time'] = np.nan

        #convert block to int
        events.block = events.block.astype({"block":int})

        # cast time fields to seconds from ms
        time_fields =['duration', 'onset',
                  'onset_delay', 'rt', 'rt_time'
                  ]
        events.loc[:, time_fields] = events.loc[:, time_fields] / 1000

        #Sort by trial number and onset time, adjust column order
        events = events.sort_values(['scan_wait_offset','onset']).reset_index(drop=True)
        cols = events.columns.tolist()
        cols.remove('onset')
        cols.remove('onset_delay')
        cols.remove('trial_number')
        cols.remove('block')
        cols = ['trial_number','block','onset','onset_delay'] + cols
        events = events[cols]

nan less than or equal to zero. pyname:onset, epname:OnsetTime, phase:Welcome, ix: 0
nan less than or equal to zero. pyname:onset_delay, epname:OnsetDelay, phase:Welcome, ix: 0
nan less than or equal to zero. pyname:duration, epname:Duration, phase:Welcome, ix: 0
nan less than or equal to zero. pyname:rt_time, epname:RTTime, phase:Welcome, ix: 0
nan less than or equal to zero. pyname:rt, epname:RT, phase:Welcome, ix: 0
nan less than or equal to zero. pyname:response, epname:RESP, phase:Welcome, ix: 0
KeyError. pyname:slider_response, epname:SliderResp, phase:Welcome, ix: 0
nan less than or equal to zero. pyname:onset, epname:OnsetTime, phase:BlockHappyPreview, ix: 0
nan less than or equal to zero. pyname:onset_delay, epname:OnsetDelay, phase:BlockHappyPreview, ix: 0
nan less than or equal to zero. pyname:duration, epname:Duration, phase:BlockHappyPreview, ix: 0
nan less than or equal to zero. pyname:rt_time, epname:RTTime, phase:BlockHappyPreview, ix: 0
nan less than or equal to zero. 

0.0 less than or equal to zero. pyname:onset_delay, epname:OnsetDelay, phase:ISIRate, ix: 19
4166.0 less than or equal to zero. pyname:duration, epname:Duration, phase:ISIRate, ix: 19
KeyError. pyname:rt_time, epname:RTTime, phase:ISIRate, ix: 19
KeyError. pyname:rt, epname:RT, phase:ISIRate, ix: 19
KeyError. pyname:response, epname:RESP, phase:ISIRate, ix: 19
KeyError. pyname:slider_response, epname:SliderResp, phase:ISIRate, ix: 19
Subtracting 44864.0 from 318000.0 (273136.0). pyname:onset, epname:OnsetTime, phase:ISITrial, ix: 19
20.0 less than or equal to zero. pyname:onset_delay, epname:OnsetDelay, phase:ISITrial, ix: 19
2000.0 less than or equal to zero. pyname:duration, epname:Duration, phase:ISITrial, ix: 19
KeyError. pyname:rt_time, epname:RTTime, phase:ISITrial, ix: 19
KeyError. pyname:rt, epname:RT, phase:ISITrial, ix: 19
KeyError. pyname:response, epname:RESP, phase:ISITrial, ix: 19
KeyError. pyname:slider_response, epname:SliderResp, phase:ISITrial, ix: 19
nan less than or

0.0 less than or equal to zero. pyname:rt_time, epname:RTTime, phase:TrialHappiness, ix: 39
0.0 less than or equal to zero. pyname:rt, epname:RT, phase:TrialHappiness, ix: 39
nan less than or equal to zero. pyname:response, epname:RESP, phase:TrialHappiness, ix: 39
nan less than or equal to zero. pyname:slider_response, epname:SliderResp, phase:TrialHappiness, ix: 39
0.0 less than or equal to zero. pyname:onset, epname:OnsetTime, phase:ISIRate, ix: 39
0.0 less than or equal to zero. pyname:onset_delay, epname:OnsetDelay, phase:ISIRate, ix: 39
5343.0 less than or equal to zero. pyname:duration, epname:Duration, phase:ISIRate, ix: 39
KeyError. pyname:rt_time, epname:RTTime, phase:ISIRate, ix: 39
KeyError. pyname:rt, epname:RT, phase:ISIRate, ix: 39
KeyError. pyname:response, epname:RESP, phase:ISIRate, ix: 39
KeyError. pyname:slider_response, epname:SliderResp, phase:ISIRate, ix: 39
Subtracting 529953.0 from 692888.0 (162935.0). pyname:onset, epname:OnsetTime, phase:ISITrial, ix: 39
19.0

0.0 less than or equal to zero. pyname:onset_delay, epname:OnsetDelay, phase:TrialHappiness, ix: 59
***********duration****************** (nan). pyname:duration, epname:Duration, phase:TrialHappiness, ix: 59
0.0 less than or equal to zero. pyname:rt_time, epname:RTTime, phase:TrialHappiness, ix: 59
0.0 less than or equal to zero. pyname:rt, epname:RT, phase:TrialHappiness, ix: 59
nan less than or equal to zero. pyname:response, epname:RESP, phase:TrialHappiness, ix: 59
nan less than or equal to zero. pyname:slider_response, epname:SliderResp, phase:TrialHappiness, ix: 59
0.0 less than or equal to zero. pyname:onset, epname:OnsetTime, phase:ISIRate, ix: 59
0.0 less than or equal to zero. pyname:onset_delay, epname:OnsetDelay, phase:ISIRate, ix: 59
2133.0 less than or equal to zero. pyname:duration, epname:Duration, phase:ISIRate, ix: 59
KeyError. pyname:rt_time, epname:RTTime, phase:ISIRate, ix: 59
KeyError. pyname:rt, epname:RT, phase:ISIRate, ix: 59
KeyError. pyname:response, epname:R

Subtracting 1007374.0 from 1353715.0 (346341.0). pyname:rt_time, epname:RTTime, phase:GetAnswer, ix: 79
533.0 less than or equal to zero. pyname:rt, epname:RT, phase:GetAnswer, ix: 79
2.0 less than or equal to zero. pyname:response, epname:RESP, phase:GetAnswer, ix: 79
KeyError. pyname:slider_response, epname:SliderResp, phase:GetAnswer, ix: 79
Subtracting 1007374.0 from 1353733.0 (346359.0). pyname:onset, epname:OnsetTime, phase:ShowChoice, ix: 79
18.0 less than or equal to zero. pyname:onset_delay, epname:OnsetDelay, phase:ShowChoice, ix: 79
4000.0 less than or equal to zero. pyname:duration, epname:Duration, phase:ShowChoice, ix: 79
KeyError. pyname:rt_time, epname:RTTime, phase:ShowChoice, ix: 79
KeyError. pyname:rt, epname:RT, phase:ShowChoice, ix: 79
KeyError. pyname:response, epname:RESP, phase:ShowChoice, ix: 79
KeyError. pyname:slider_response, epname:SliderResp, phase:ShowChoice, ix: 79
Subtracting 1007374.0 from 1357745.0 (350371.0). pyname:onset, epname:OnsetTime, phase:Sho

## Code from Dylan

In [ ]:
event_meta = {
    'cue_type':{'Description': "What type of cue is bing presented"},
    'current_mean_accuracy':{'Description': "Accuracy as of the end of this trial as a fraction between 0 and 1"},
    'duration': {'Description':'Time in miliseconds that this phase of the trial is presented'},
    'duration_error': {'Description':'Milliseconds of error in the duration'},
    'is_hit': {'Description':'Did the subject respond in time on this trial'},
    'isi_adjusted': {'Description':'Calculated ISI to follow this trial'},
    'onset':{'Description': "Time the draw command on the monitor starts in miliseconds"},
    'onset_delay':{'Description': "Difference between onset time and intended display time."},
    'response':{'Description': "Value of the key response. 5.0 during the Feedback phase represents the scanner sync pulse."},
    'response_type':{'Description': "How was their response categorized. One of ['NoResponse', 'GoodResponse', 'PrematurePress', 'LatePress']"},
    'reward': {'Description': "Number of points recieved in this trial"},
    'rt': {'Description': "Reaction time in miliseconds. During the Feedback phase this is the time between the start of the phase and receiving the next sync pulse"},
    'rt_time': {'Description': "Time of the response in miliseconds since the start of the scan"},
    'total_reward': {'Description': "Total reward in points as of the end of this trial."},
    'trial_type': {'Description': 'Categorization of the trial on the basis of the cue type and the result'},
    'trial_phase': {'Description': 'Which phase of the trial does this event represent'},
    'one_press': {'Description': 'Did the participant hit the button once and only once during this trial'},
}
# Regex for pulling ScanWait.OffsetTime from eprime text file
swo_pat = re.compile("ScanWait\.OffsetTime: (?P<res>[0-9]+)")
ev_types = ['Cue', 'Fixation', 'Target', 'Blank', 'Feedback', 'ISIslide']
time_types = {'onset': 'OnsetTime',
              'onset_delay': 'OnsetDelay',
              'duration': 'Duration',
              'duration_error': 'DurationError',
              'rt_time': 'RTTime',
              'rt': 'RT',
              'response': 'RESP'}
# Which times need to be adjusted based on the ScanWait OffsetTime
adj_times = ['onset', 'rt_time']
i = 0
for ix,row in mid_scans_merged.loc[mid_scans_merged.md5sum.notnull(),:].iterrows():
    task_events = pd.read_csv(row.path)
# #     if row.path == '/EDB/MBDU/to_bids_beh_landing/General folder/MID1-23270-3.csv':
# #         task_events['MID_file_path'] = "Y:/Data/fMRI_Behavioral_Task_Data/tasks2/MID1/MID1-23720-4.txt"
#     # Get Eprime text files to rerieve the ScanWait.OffsetTime
#     try:
#         ep_text_path = task_events.MID_file_path.unique()
#     except AttributeError:
#         ep_text_path = task_events.MID_file_Visits_path.unique()
#     assert len(ep_text_path) == 1
#     ep_text_path = Path(ep_text_path[0].replace('Y:/', sdan1_path))
#     ep_dat = ep_text_path.read_text(encoding='utf-16')
    # Convert back to miliseconds
    scanwait_offsettime = row.scanwait_offsettime * 1000
    events = []
    task_events = task_events.loc[task_events.TrialCategory != "999", :]
    task_events = task_events.reset_index().rename(columns={'index': 'trial_number'})
    for ix, te_row in task_events.iterrows():
        for tt in ev_types:
            ev = {}
            for pyname, epname in time_types.items():
                try:
                    if (pyname in adj_times) & (te_row[f'{tt}.{epname}'] != 0):
                        ev[pyname] = te_row[f'{tt}.{epname}']-scanwait_offsettime
                    else:
                        ev[pyname] = te_row[f'{tt}.{epname}']
                except KeyError:
                    pass
            ev['trial_phase'] = tt
            #ev['sample'] = row.Sample
            ev['cue_type'] = te_row.Cue
            ev['response_type'] = te_row.ResponseType
            ev['is_hit'] = te_row.IsHit
            ev['trial_type'] = te_row.TrialCategory
            ev['reward'] = te_row.Reward
            ev['total_reward'] = te_row.TotalReward
            ev['current_mean_accuracy'] = float(te_row.CurrentMeanAccuracy[1:].replace('%',''))/100
            ev['isi_adjusted'] = te_row.ISIAdjusted
            ev['trial_number'] = te_row.trial_number
            events.append(ev)
    events = pd.DataFrame(events)
    events.loc[events.rt_time== 0, 'rt'] = np.nan
    events.loc[events.rt_time== 0, 'rt_time'] = np.nan
    # add in stimulus type tags for stimuli of interest for modeling
#     cues = events.loc[events.trial_phase == "Cue", :]
#     target = events.loc[events.trial_phase == "Target", ['trial_number', 'onset']]
#     cf = cues.merge(target, how='left', on='trial_number', suffixes=['_cue', '_target'])
#     cf['onset'] = cf.onset_cue
#     cf['duration'] = cf.onset_target - cf.onset_cue
#     cf['stimulus_type'] = np.nan
#     cf.loc[cf.cue_type == "Control", 'stimulus_type'] = "neutral_anticipation"
#     cf.loc[cf.cue_type == "Lose2", 'stimulus_type'] = "loss_anticipation"
#     cf.loc[cf.cue_type == "Win2", 'stimulus_type'] = "gain_anticipation"
#     cf['trial_phase'] = 'Cue + Fixation'
#     cf = cf.drop(["onset_target", "onset_cue"], axis=1)
#     events['stimulus_type'] = np.nan
#     events.loc[(events.trial_phase == "Feedback") & (events.trial_type == "Neutral"), 'stimulus_type'] = 'neutral_feedback'
#     events.loc[(events.trial_phase == "Feedback") & (events.trial_type == "AvoidedLoss"), 'stimulus_type'] = 'avoided_loss_feedback'
#     events.loc[(events.trial_phase == "Feedback") & (events.trial_type == 'Loss'), 'stimulus_type'] = 'loss_feedback'
#     events.loc[(events.trial_phase == "Feedback") & (events.trial_type == 'Win'), 'stimulus_type'] = 'gain_feedback'
#     events.loc[(events.trial_phase == "Feedback") & (events.trial_type == 'MissedWin'), 'stimulus_type'] = 'missed_gain_feedback'
#     events = pd.concat([events, cf], sort=False).sort_values(['trial_number', 'onset']).reset_index(drop=True)
#     #events = events.loc[pd.notnull(events.stimulus_type), :]
#     events['stimulus_type'] = events.stimulus_type.fillna(events.trial_type + '_' + events.trial_phase)
    # Add button press events
    motor = events.query('response != 5 & rt_time.notnull()').copy(deep=True)
    motor['onset'] = motor['rt_time']
    motor['rt_time'] = np.nan
    motor['duration'] = 300
    motor['trial_phase'] = 'ButtonPress'
    motor = motor.reset_index()
    events = pd.concat([events, motor], ignore_index=True, sort=False ).sort_values(['trial_number', 'onset']).reset_index(drop=True)
    # cast time fields to seconds from ms
    time_fields =['duration', 'duration_error',
              'isi_adjusted', 'onset',
              'onset_delay', 'rt', 'rt_time',
              ]
    events.loc[:, time_fields] = events.loc[:, time_fields] / 1000
    events = events.set_index(['onset', 'duration']).reset_index()
    bpdf = pd.DataFrame(events.groupby(['trial_number']).apply(lambda df: df.trial_phase.isin(['ButtonPress']).sum()), columns=['nbuttonpress']).reset_index()
    bpdf['one_press'] = bpdf.nbuttonpress == 1
    events = events.merge(bpdf.loc[:, ['trial_number', 'one_press']], how="left", on="trial_number")
    # Reaction time sanity check
    math_ind = np.isclose(events.rt_time, (events.onset + events.rt))
    assert (events.rt_time.notnull() & ~math_ind).sum() == 0
    for ds in ['longitudinal', 'outpatient', 'inpatient', 'other']:
        if pd.notnull(row[f'{ds}_bids_path']):
            events_file = row[f'{ds}_bids_path'].replace('_bold', '_events.tsv')
            event_meta_file = Path(row[f'{ds}_bids_path'].replace('_bold', '_events.json'))
            assert(str(events_file) != '')
            assert(str(event_meta_file) != '' )
            print(events_file)
            print(event_meta_file)
            events.to_csv(events_file, index=False, sep='\t', na_rep='n/a')
            event_meta_file.write_text(json.dumps(event_meta, indent=2))

## Past Versions

In [54]:
#First draft

#Review these event types
ev_types = ['Welcome', 'BlockHappyPreview', 'BlockStartHappiness','ScanWait', 'GetReady', 'GetAnswer', 'ShowChoice', 'ShowResult', 'TrialHappiness','ISIRate','ISITrial', 'CurrentEarnings', 'LifeHappyPreview', 'LifeHappiness', 'Rest', 'Goodbye']
time_types = {'onset': 'OnsetTime',
              'onset_delay': 'OnsetDelay',
              'duration': 'Duration',
              'duration_error': 'DurationError',
              'rt_time': 'RTTime',
              'rt': 'RT',
              'response': 'RESP',
              'slider_response': 'SliderResp'
             }

events = []
for ix, te_row in task_events.iterrows():
    for tt in ev_types:
        ev = {}
        for pyname, epname in time_types.items():
            try: 
                ev[pyname] = te_row[f'{tt}__{epname}']
            except KeyError:
                #Calculates duration of response phases using last slider motion
                #Should onset time be SliderResp or regular onset for these phases?
                if pyname == 'duration':
                    ev[pyname] = te_row[f'{tt}__Slider_LastMotion'] - te_row[f'{tt}__SliderRespOnsetTime']
                pass
        #check these. Is idff a time value?
        try:
            ev['trial_number'] = int(te_row['TrialList__Sample'])
        except ValueError:
            break
        ev['trial_phase'] = tt
        ev['trial_type'] = te_row['TrialType']
        ev['rpe'] = te_row['RPE']
        ev['outcome'] = te_row['Outcome']
        ev['outcome1_amount'] = te_row['Outcome1Amount']
        ev['outcome2_amount'] = te_row['Outcome2Amount']
        ev['certain_amount'] = te_row['CertainAmount']
        ev['current_amount'] = te_row['CurrentAmount']
        ev['amount_change'] = te_row['AmountChange']
        ev['rating'] = te_row['DoRating']
        ev['multiplier'] = te_row['Multiplier']
        ev['question_duration'] = te_row['QuestionDur']
        ev['mood_target'] = te_row['MoodTarget']
        events.append(ev)    
        
events = pd.DataFrame(events)
#drop rows without an onset and trial number, aka nonevent rows
events.dropna(subset=['trial_number','onset'], inplace=True)

#convert reaction times of 0 to NaN
events.loc[events.rt_time== 0, 'rt'] = np.nan
events.loc[events.rt_time== 0, 'rt_time'] = np.nan

# cast time fields to seconds from ms
time_fields =['duration', 'duration_error', 'onset',
          'onset_delay', 'rt', 'rt_time'
          ]
events.loc[:, time_fields] = events.loc[:, time_fields] / 1000
events = events.sort_values(['trial_number','onset']).reset_index(drop=True)

cols = events.columns.tolist()
cols.remove('onset')
cols.remove('onset_delay')
cols.remove('trial_number')
cols = ['trial_number','onset','onset_delay'] + cols
events = events[cols]

In [300]:
#Second draft
#def eprime_to_bids(fd):
    for file in os.listdir(fd + 'MMI_events/'):
        #iterates through all csv and excel files
        if '.csv' in file:
            task_events = pd.read_csv(f'{fd}MMI_events/{file}')
        elif '.x' in file:
            task_events = pd.read_excel(f'{fd}MMI_events/{file}')
        else:
            print("Unable to read events file.")
            
        sub = file.split('-')[1].split('_')[0]
            
        #Review these event types
        ev_types = ['Welcome', 'BlockHappyPreview', 'BlockStartHappiness','ScanWait', 'GetReady', 'GetAnswer', 'ShowChoice', 'ShowResult', 'TrialHappiness','ISIRate','ISITrial', 'CurrentEarnings', 'LifeHappyPreview', 'LifeHappiness', 'Rest', 'Goodbye']
        time_types = {'onset': 'OnsetTime',
                      'onset_delay': 'OnsetDelay',
                      'duration': 'Duration',
                      'rt_time': 'RTTime',
                      'rt': 'RT',
                      'response': 'RESP',
                      'slider_response': 'SliderResp'
                     }
        
        #getting scan wait offset times for each block to subtract from onset and reaction times within the indices of each block
        adj_times = ['onset', 'rt_time']     
        scan_wait_offsets = []
        for ix, off_row in task_events[np.isnan(task_events.ScanWait__OffsetTime) == False].iterrows():
            scan_wait_offsets.append(off_row.ScanWait__OffsetTime)
        
        events = []
        block = 0
        for ix, te_row in task_events.iterrows():
            #sets scan offset time for each block, increments when the next block begins
            if ix > scan_wait_offsets[block][0] and block < 2:
                block += 1
            for phase in ev_types:
                ev = {}
                for pyname, epname in time_types.items():
                    try: 
#                         #subtract offset times from nonzero onset and reaction times
#                         if pyname in adj_times and te_row[f'{phase}__{epname}'] > 0:
#                             ev[pyname] = te_row[f'{phase}__{epname}'] - scan_wait_offsets[block][1]
#                         else:
                            ev[pyname] = te_row[f'{phase}__{epname}']
                    except KeyError:
                        #Calculates duration of response phases using last slider motion
                        #Should onset time be SliderResp or regular onset for these phases?
                        if pyname == 'duration':
                            ev[pyname] = te_row[f'{phase}__Slider_LastMotion'] - te_row[f'{phase}__SliderRespOnsetTime']
                        pass
                #Adds trial number from TrialList Sample, converts float to int 
                try:
                    ev['trial_number'] = int(te_row['TrialList__Sample'])
                except ValueError:
                    pass
                ev['trial_phase'] = phase
                ev['trial_type'] = te_row['TrialType']
                ev['rpe'] = te_row['RPE']
                ev['outcome'] = te_row['Outcome']
                ev['outcome1_amount'] = te_row['Outcome1Amount']
                ev['outcome2_amount'] = te_row['Outcome2Amount']
                ev['certain_amount'] = te_row['CertainAmount']
                ev['current_amount'] = te_row['CurrentAmount']
                ev['amount_change'] = te_row['AmountChange']
                ev['rating'] = te_row['DoRating']
                ev['multiplier'] = te_row['Multiplier']
                ev['question_duration'] = te_row['QuestionDur']
                ev['mood_target'] = te_row['MoodTarget']
                events.append(ev)    

        events = pd.DataFrame(events)
        
#         #drop rows without an onset (or if onset is 0)
#         events = events[events.onset != 0]
#         events.dropna(subset=['trial_number','onset'], inplace=True)

#         #convert reaction times of 0 to NaN
#         events.loc[events.rt_time== 0, 'rt'] = np.nan
#         events.loc[events.rt_time== 0, 'rt_time'] = np.nan
        
#         # cast time fields to seconds from ms
#         time_fields =['duration', 'onset',
#                   'onset_delay', 'rt', 'rt_time'
#                   ]
#         events.loc[:, time_fields] = events.loc[:, time_fields] / 1000

        #Sort by trial number and onset time, adjust column order
        events = events.sort_values(['onset','trial_number']).reset_index(drop=True)
        cols = events.columns.tolist()
        cols.remove('onset')
        cols.remove('onset_delay')
        cols.remove('trial_number')
        cols = ['trial_number','onset','onset_delay'] + cols
        events = events[cols]
        
        return events
        #events.to_csv(f'{fd}BIDS_events/BIDS_MMI_events_{sub}.tsv',sep='\t',index=False)